#1.Mount Drive

In [1]:
from google.colab import drive # access data in Google Drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#2. Densemap Net

2.1 Import libaries

In [0]:
import time

class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self, sec):
        if sec < 0:
            sec = "%0.2f" % (sec * 100)
            return sec + " msec"
        elif sec < 60:
            sec = "%0.4f" % sec
            return sec + " sec"
        elif sec < (60 * 60):
            sec = "%0.4f" % (sec / 60)
            return sec + " min"
        else:
            sec = "%0.4f" % (sec / (60 * 60))
            return sec + " hr"
    def elapsed_time(self):
        delta = time.time() - self.start_time
        return delta

    def print_elapsed_time(self):
        print("Speed: %s " % self.elapsed(self.elapsed_time()))

class Settings(object):
    def __init__(self):
        self.ydim = 540
        self.xdim = 960
        self.channels = 3        
        self.model_weights = None






#2.2 model

In [27]:

import matplotlib.pyplot as plt
#from keras.callbacks import *
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import imageio
import keras
from keras.layers import Dense, Dropout
from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.layers import ZeroPadding2D, BatchNormalization, Activation
from keras.layers import UpSampling2D 
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.models import load_model, Model
from keras.layers.pooling import MaxPooling2D
from keras.utils import plot_model

import numpy as np
#from utils import Settings


class DenseMapNet(object):
    def __init__(self, settings):
        self.settings =settings
        self.xdim = self.settings.xdim 
        self.ydim = self.settings.ydim
        self.channels = self.settings.channels
        self.model = None

    def build_model(self, lr=1e-3):
        dropout = 0.2
      
        shape=(None,540, 960,3)#self.ydim, self.xdim, self.channels)
        left = Input(batch_shape=shape)
        right = Input(batch_shape=shape)
        print(shape)
        # left image as reference
        x = Conv2D(filters=16, kernel_size=5, padding='same')(left)
        xleft = Conv2D(filters=1,
                       kernel_size=5,
                       padding='same',
                       dilation_rate=2)(left)

        # left and right images for disparity estimation
        xin = keras.layers.concatenate([left, right])
        xin = Conv2D(filters=32, kernel_size=5, padding='same')(xin)

        # image reduced by 8
        x8 = MaxPooling2D(8)(xin)
        x8 = BatchNormalization()(x8)
        x8 = Activation('relu', name='downsampled_stereo')(x8)

        dilation_rate = 1
        y = x8
        # correspondence network
        # parallel cnn at increasing dilation rate
        for i in range(4):
            a = Conv2D(filters=32,
                       kernel_size=5,
                       padding='same',
                       dilation_rate=dilation_rate)(x8)
            a = Dropout(dropout)(a)
            y = keras.layers.concatenate([a, y])
            dilation_rate += 1

        dilation_rate = 1
        x = MaxPooling2D(8)(x)
        # disparity network
        # dense interconnection inspired by DenseNet
        for i in range(4):
            x = keras.layers.concatenate([x, y])
            y = BatchNormalization()(x)
            y = Activation('relu')(y)
            y = Conv2D(filters=64,
                       kernel_size=1,
                       padding='same')(y)

            y = BatchNormalization()(y)
            y = Activation('relu')(y)
            y = Conv2D(filters=16,
                       kernel_size=5,
                       padding='same',
                       dilation_rate=dilation_rate)(y)
            y = Dropout(dropout)(y)
            dilation_rate += 1
        
        # disparity estimate scaled back to original image size
        x = keras.layers.concatenate([x, y], name='upsampled_disparity')
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(filters=32, kernel_size=1, padding='same')(x)
        x = UpSampling2D(8)(x)
        if not self.settings.nopadding:
            x = ZeroPadding2D(padding=(2, 0))(x)

        # left image skip connection to disparity estimate
        x = keras.layers.concatenate([x, xleft])
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(filters=16, kernel_size=5, padding='same')(y)

        x = keras.layers.concatenate([x, y])
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2DTranspose(filters=1, kernel_size=9, padding='same')(y)

        # prediction

        yout = Activation('sigmoid', name='disparity_output')(y)

        # densemapnet model
        self.model = Model([left, right],yout)
       
        if self.settings.model_weights:
            print("Loading checkpoint model weights %s...."
                  % self.settings.model_weights)
            self.model.load_weights(self.settings.model_weights)



        self.model.compile(loss='mse', optimizer=Adam())

        print("DenseMapNet Model:")
        self.model.summary()
        plot_model(self.model, to_file='densemapnet.png', show_shapes=True)

        return self.model
print('model loaded')

model loaded


# 2.3 arrange settings and run network

In [28]:


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import random
import keras
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.optimizers import RMSprop, SGD,Adam
import tensorflow as tf
import numpy as np
from keras import backend as K
import argparse
import os
from os import path
import time
import matplotlib.image as img
import matplotlib.pyplot as plt
from scipy import misc


class Predictor(object):
    def __init__(self, settings=Settings()):
        settings.nopadding = False
        self.settings = settings
        self.mkdir_images()
        
        self.get_max_disparity()
        self.load_test_data()
        
        self.network  = None
        self.train_data_loaded = False
        self.train_lx=np.full((144,540, 960, 3),0).astype('float32')
        self.train_rx=np.full((144, 540, 960, 3),0).astype('float32')
        self.train_dx=np.full((144,540, 960, 1),0).astype('float32')
        self.predicted=np.full((17,540, 960, 1),0).astype('float32')
        if self.settings.epe:
            self.best_epe = self.settings.epe
        else:
            self.best_epe = 100.0

 

    def load_test_disparity(self):
        filename =os.path.join(self.settings.pdir, "testD.npz")
        print("Loading... ", filename)
        self.test_dx=np.load( filename)['arr_0']

        self.dmax =  max(self.dmax, np.amax(self.test_dx))
        self.dmin =  min(self.dmin, np.amin(self.test_dx))
        self.sum += np.sum(self.test_dx, axis=0)
        self.size += self.test_dx.shape[0]
        print("Max disparity on entire dataset: ", self.dmax)
        print("Min disparity on entire dataset: ", self.dmin)
        if self.settings.predict:
            filename =os.path.join(self.settings.pdir, "testD.npz" )
            print("Loading... ", filename)
            self.train_dx=np.load( filename)['arr_0']

            

        dim = self.test_dx.shape[1] * self.test_dx.shape[2]
        self.ave = np.sum(self.sum / self.size) / dim
        print("Ave disparity: ", self.ave)
        self.test_dx = self.test_dx.astype('float32') / self.dmax
        print("Scaled disparity max: ", np.amax(self.test_dx))
        print("Scaled disparity min: ", np.amin(self.test_dx))


        shape = [-1, self.test_dx.shape[1], self.test_dx.shape[2], 1]
        self.test_dx = np.reshape(self.test_dx, shape)

    def get_max_disparity(self):
        self.dmax = 0
        self.dmin = 255
        self.sum = None
        self.size = 0
        filename =os.path.join(self.settings.pdir, "dataD.npz")
        print(filename)
     
        
        Data=np.load( filename)
       
        count =  1

        filename =os.path.join(self.settings.pdir, "dataD.npz")
        print("Loading... ", filename)
#             
        self.train_dx=np.load( filename)['arr_0']

        self.dmax =  max(self.dmax, np.amax(self.train_dx))
        self.dmin =  min(self.dmin, np.amin(self.train_dx))
        if self.sum is None:
          self.sum = np.sum(self.train_dx, axis=0)
        else:
          self.sum += np.sum(self.train_dx, axis=0)
        self.size += self.train_dx.shape[0]
        self.load_test_disparity()

    def load_test_data(self):
        filename =os.path.join(self.settings.pdir, "testL.npz")
        print("Loading... ", filename)
        self.test_lx=np.load( filename)['arr_0']
        self.test_lx=self.test_lx/np.max(self.test_lx)

            
        filename =os.path.join(self.settings.pdir, "testR.npz" )
        print("Loading... ", filename)
        self.test_rx=np.load( filename)['arr_0']
        self.test_rx=self.test_rx/np.max(self.test_rx)



        self.channels = self.settings.channels = self.test_lx.shape[3]
        self.xdim = self.settings.xdim = self.test_lx.shape[2]
        self.ydim = self.settings.ydim = self.test_lx.shape[1]

    def load_train_data(self, index):

        print(index)
        filename =os.path.join(self.settings.pdir, "dataL.npz")
        print("Loading... ", filename)
        self.train_lx=np.load( filename)['arr_0']
        self.train_lx=self.train_lx/np.max(self.train_lx)
        filename =os.path.join(self.settings.pdir, "dataR.npz")
        print("Loading... ", filename)
        train_rx=np.load( filename)['arr_0']
        self.train_rx=self.train_rx/np.max(self.train_rx)
        filename =os.path.join(self.settings.pdir, "dataD.npz") 
        tempDisp=np.load( filename)['arr_0']
        tempDisp = tempDisp.astype('float32') / self.dmax
        print("Scaled disparity max: ", np.amax(tempDisp))
        print("Scaled disparity min: ", np.amin(tempDisp))
        print(tempDisp.shape)
        self.train_dx =tempDisp       
        self.channels = self.settings.channels = 3
        self.xdim = self.settings.xdim = tempDisp.shape[2]
        self.ydim = self.settings.ydim = tempDisp.shape[1]
        self.train_data_loaded = True

    def train_network(self):
        self.train_all()
         



    def train_all(self, epochs=200, lr=1e-3):
        checkdir = "checkpoint"
        try:
            os.mkdir(checkdir)
        except FileExistsError:
            print("Folder exists: ", checkdir)
        

        checkpoint = ModelCheckpoint(filepath=os.path.join(self.settings.Weights_pdir,'weights{epoch:03d}.h5'),
                                     save_weights_only=True,
                                     verbose=1,
                                     save_best_only=False)
        predict_callback = LambdaCallback(on_epoch_end=lambda epoch,
                                          logs: self.predict_disparity(epoch=epoch))
        
        Callbacks = [checkpoint, predict_callback]
        self.load_train_data(1)
        if self.network is None:
            self.network = DenseMapNet(settings=self.settings)
            self.model = self.network.build_model(lr=lr)
        print("Using loss=crossent on sigmoid output layer")
        self.model.compile(loss=self.settings.loss_function, optimizer=Adam())

        if self.settings.model_weights:
            if self.settings.notrain:
                self.predict_disparity()
                return

        
        
        print(self.train_lx.shape)
        x = [self.train_lx, self.train_rx]
        x_test=[self.test_lx, self.test_rx]
        hist=self.model.fit(x,
                       self.train_dx,
                       validation_data=(x_test, self.test_dx),
                       epochs=self.settings.epochs,
                       batch_size=4,
                       shuffle=True,
                       callbacks=[checkpoint, predict_callback],
                            verbose=1)
        Path_weight=os.path.join(self.settings.Weights_pdir,'Final_weight.h5')
        print(Path_weight)
        self.model.save_weights(Path_weight)
    

    def mkdir_images(self):
        pdir = self.settings.images_pdir

        for dirname in ["train", "test"]:
            cdir = os.path.join(pdir, dirname)
            filepath = os.path.join(cdir, "left")
            os.makedirs(filepath, exist_ok=True)
            filepath = os.path.join(cdir, "right")
            os.makedirs(filepath, exist_ok=True)
            filepath = os.path.join(cdir, "disparity")
            os.makedirs(filepath, exist_ok=True)
            filepath = os.path.join(cdir, "prediction")
            os.makedirs(filepath, exist_ok=True)


    def get_epe(self, use_train_data=False, get_performance=True, epoch=77):
        if use_train_data:
            lx = self.train_lx
            rx = self.train_rx
            dx = self.train_dx
            print("Using train data... Size: ", lx.shape[0])
        else:
            lx = self.test_lx
            rx = self.test_rx
            dx = self.test_dx

            if self.settings.predict:
                print("Using complete data... Size: ", lx.shape[0])
            else:
                print("Using test data... Size: ", lx.shape[0])

        # sum of all errors (normalized)
        epe_total = 0
        # count of images
        t = 0
        nsamples = lx.shape[0]
        elapsed_total = 0.0
        if self.settings.images:
            print("Saving images on folder...")
        for i in range(0, nsamples, 1):
            indexes = np.arange(i, i + 1)
            left_images = lx[indexes, :, :, : ]
            right_images = rx[indexes, :, :, : ]
            disparity_images = dx[indexes, :, :, : ]

            # measure the speed of prediction on the 10th sample to avoid variance
            if get_performance:
                start_time = time.time()
                predicted_disparity = self.model.predict([left_images, right_images])
                elapsed_total += (time.time() - start_time)
            else:
                predicted_disparity = self.model.predict([left_images, right_images])

            self.predicted[indexes,:,:,:] = predicted_disparity[0, :, :, :]
            predicted= predicted_disparity[0, :, :, :]
           
            ground = disparity_images[0, :, :, :]

            dim = predicted.shape[0] * predicted.shape[1]
            epe = predicted - ground
            # normalized error on all pixels
            epe = np.sum(np.absolute(epe))
            
            epe = epe / dim
            epe_total += epe
            if (epoch%10) == 0:

                path = "test"
                if use_train_data:
                    path = "train"
                filepath  = os.path.join(self.settings.images_pdir, path)
                left = os.path.join(filepath, "left")
                right = os.path.join(filepath, "right")
                disparity = os.path.join(filepath, "disparity")
                prediction = os.path.join(filepath, "prediction")
                print(prediction)
                filename = 'Image{0}.png'.format(epoch)
                left = os.path.join(left, filename)
                if left_images[0].shape[2] == 1:
                    self.predict_images(left_images[0], left)
                else:
                    plt.imsave(left, left_images[0])

                right = os.path.join(right, filename)
                if right_images[0].shape[2] == 1:
                    self.predict_images(right_images[0], right)
                else:
                    plt.imsave(right, right_images[0])
                self.predict_images(predicted, os.path.join(prediction, filename))
                self.predict_images(ground, os.path.join(disparity, filename))

        epe = epe_total / nsamples 
        # epe in pix units
        epe = epe * self.dmax

        print("EPE: %0.2fpix" % epe)
        if epe < self.best_epe:
            self.best_epe = epe
            print("------------------- BEST EPE : %f ---------------------" % epe)
            tmpdir = "tmp"
            try:
                os.mkdir(tmpdir)
            except FileExistsError:
                a=3#print("Folder exists: ", tmpdir)
            filename = open('tmp/epe.txt', 'a')
            datetime = time.strftime("%H:%M:%S")
            filename.write("%s : EPE: %f\n" % (datetime,  epe))
            filename.close()
        # speed in sec
        if get_performance:
            print("Speed: %0.4fsec" % (elapsed_total / nsamples))
            print("Speed: %0.4fHz" % (nsamples / elapsed_total))

    def predict_images(self, image, filepath):
        size = [image.shape[0], image.shape[1]]

        image =  np.clip(image, 0.0, 1.0)
        image *= 255

        image = image.astype(np.uint8)
        
        image = np.reshape(image, size)
        imageio.imwrite(filepath, image)

    def predict_disparity(self,epoch=777):
        if self.settings.predict:
            if self.network is None:
                self.network = DenseMapNet(settings=self.settings)
                self.model = self.network.build_model()
            # gpu is slow in prediction during initial load of data
            # distorting the true speed of the network
            # we get the speed after 1 prediction
            if self.settings.images:
                self.get_epe(use_train_data=False, get_performance=True, epoch=epoch)
            else:
                for i in range(4):
                    self.get_epe(use_train_data=False, get_performance=True, epoch=epoch)
        else:
            # self.settings.images = True
            self.get_epe(use_train_data=False, get_performance=True, epoch=epoch)
            # self.get_epe(use_train_data=False)
            if self.settings.notrain:
                self.get_epe()


if __name__ == '__main__':

    
    
    settings = Settings()

    settings.pdir="/content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset"
    #define if you want to train
    settings.notrain = False
    settings.loss_function="mse"
    settings.epochs=300                      
    #insert Model Weights (for predict)
    settings.model_weights =""#"/content/gdrive/My Drive/Stereo_1/DenseMapNet/DataFamily/CosHweights285.h5"
    
    #directory to save weights
    settings.Weights_pdir= "/content/gdrive/My Drive/Stereo_1/DenseMapNet/DataFamily/Bin_Weights"

    settings.num_dataset = 1
    settings.predict = False
    
    #Mark if you want to export images and if so, to which directory
    settings.images = True 
    settings.images_pdir = "/content/gdrive/My Drive/Stereo_1/DenseMapNet/DataFamily/Bin/images"
    

    settings.epe = True
   

    
    

    predictor = Predictor(settings=settings)
    if settings.predict:
        predictor.predict_disparity()
        Output=predictor.predicted
    else:
        predictor.train_network()


/content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/dataD.npz
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/dataD.npz
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/testD.npz
Max disparity on entire dataset:  0.0045319297
Min disparity on entire dataset:  1.5259022e-05
Ave disparity:  0.0006912775981573411
Scaled disparity max:  0.9898989
Scaled disparity min:  0.0033670033
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/testL.npz
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/testR.npz
1
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/dataL.npz
Loading...  /content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/dataR.npz


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: RuntimeWarning: invalid value encountered in true_divide


Scaled disparity max:  1.0
Scaled disparity min:  0.0033670033
(144, 540, 960, 1)
(None, 540, 960, 3)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
DenseMapNet Model:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 540, 960, 3)  0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 540, 960, 3)  0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 540, 960, 6)  0           input_11[0][0]                   
                                                                 input_

KeyboardInterrupt: ignored

#3.JF filter

3.1Import

In [2]:
import os # os specific actions
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # image processing
import matplotlib.pyplot as plt # plots
import random
# deep learning framework
import keras.backend as K
from keras import utils
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam,RMSprop
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
from keras import initializers
#patches
from skimage import data, util
from skimage.measure import label, regionprops
import matplotlib.patches as mpatches
from PIL import Image
from sklearn.feature_extraction import image

Using TensorFlow backend.


#3.2 define settings

In [22]:
activFunc=ELU()
Dropout_active=False
DO_val=0.5

Create_train_data=True
Train_size=50000
Test_size=5000
#for External inputs (for internal, just put in Guide,Targe and GroundTruth the correct arrays)
Guide_filePath ="/content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/testL.npz"
Guide=np.load( Guide_filePath)['arr_0']
Target_filePath="/content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/TestOutput.npz"
Target=np.load(Target_filePath)['arr_0']
GRoundTruth_filePath ="/content/gdrive/My Drive/DeepLearning/Project_submission/DenseMap/Dataset/testD.npz"
GroundTruth=np.load( GRoundTruth_filePath)['arr_0']

if not Create_train_data:
  Guide_train_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/Train64Guide.npz'
  Guide_test_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/Test64Guide.npz'
  Target_train_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/Train64Target.npz'
  Target_test_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/Test64Target.npz'
  GRoundTruth_train_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/Train64OutPut.npz'
  GRoundTruth__test_filePath='/content/gdrive/My Drive/Stereo_2/JF_dataset/TestOut64Put.npz'



predict=False
GuidePath2Predict ="/content/gdrive/My Drive/Stereo_2/Data/testL.npz"
GT2Predict ="/content/gdrive/My Drive/Stereo_2/Data/testD.npz"
TargetPath2Predict ="/content/gdrive/My Drive/Stereo_2/Data/TestOutput.npz"

path_save_weight='/content/gdrive/My Drive/Stereo_2/test/moo.h5'
path_save_weight_checkPoint='/content/gdrive/My Drive/Stereo_2/test/moo2.h5'
path_load_weight='/content/gdrive/My Drive/DeepLearning/Project_submission/JF_Net/Weights/MSE/JF_MSE_ELU_NO_DO_WEIGHT10.h5'




(17, 540, 960, 3)


#3.3 Create Train data

In [24]:




GroundTruth=1-GroundTruth
GroundTruth =  np.clip(GroundTruth, 0.0, 1.0)
GroundTruth *= 255
T_train=np.full((Train_size,64, 64, 3),0).astype('float32')
G_train=np.full((Train_size,64, 64, 3),0).astype('float32')
Y_train=np.full((Train_size,64, 64, 1),0).astype('float32')
G_test=np.full((Test_size,64, 64, 3),0).astype('float32')
T_test=np.full((Test_size,64, 64, 3),0).astype('float32')
Y_test=np.full((Test_size,64, 64, 1),0).astype('float32')
if Create_train_data:
      cal=0
      cal2=0
      while (cal!=Train_size):
                
                ind= random.randint(4,16)        
                A=GroundTruth[ind,:,:,0]
                i = random.choice([m for m in range(A.shape[0])])
                j = random.choice([m for m in range(A.shape[1])])

                if j-32>0 and i-32>0 and j<A.shape[0]-32 and i<A.shape[1]-32 :# and  Edge_A[j][i]==1:

                    Y_train[cal,:,:,0] =GroundTruth[ind,j-32:j+32, i-32:i+32,0]
                    G_train[cal,:,:,:]=Guide[ind,j-32:j+32, i-32:i+32,0:3]/np.max(Guide)
                    T_train[cal,:,:,0]=Target[ind,j-32:j+32, i-32:i+32,0]/np.max(Target)

                    cal=cal+1

      while (cal2!=Test_size):
                
                ind= random.randint(0,3)        
                A=GroundTruth[ind,:,:,0]
                i = random.choice([m for m in range(A.shape[0])])
                j = random.choice([m for m in range(A.shape[1])])

                if j-32>0 and i-32>0 and j<A.shape[0]-32 and i<A.shape[1]-32 :# 

                    Y_test[cal2,:,:,0] =GroundTruth[ind,j-32:j+32, i-32:i+32,0]
                    G_test[cal2,:,:,:]=Guide[ind,j-32:j+32, i-32:i+32,0:3]/np.max(Guide)
                    T_test[cal2,:,:,0]=Target[ind,j-32:j+32, i-32:i+32,0]/np.max(Target)
                    cal2=cal2+1
                    
if not Create_train_data:

  T_train=np.full((Train_size,64, 64, 1),0).astype('float32')
  T_test=np.full((Test_size,64, 64, 1),0).astype('float32')
  G_train=np.load(Guide_train_filePath)['arr_0']
  Y_train=np.load(GRoundTruth_train_filePath)['arr_0']
  temp=np.load(Target_train_filePath)['arr_0']
  T_train[:,:,:,0]=temp[:,:,:,0]

  G_test=np.load(Guide_test_filePath)['arr_0']
  Y_test=np.load(GRoundTruth__test_filePath)['arr_0']
  temp=np.load(Target_test_filePath)['arr_0']
  T_test[:,:,:,0]=temp[:,:,:,0]

0
0
0
0
1
1
1
2
2
3
3
4
5
6
7
7
8
8
8
9
9
9


#3.4 Build Model

In [13]:
# Define model


input_layer_target = Input(shape=T_train.shape[1:]) # full size mask as GT
input_layer_guide = Input(shape=G_train.shape[1:])   # full size image

print(input_layer_target.shape)
print(input_layer_guide.shape)

# CNN_T

# 1
T1 = Conv2D(filters=96, kernel_size=(9,9), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(input_layer_target)

T1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(T1)
if Dropout_active:
  T1 = Dropout(DO_val)(T1)

# 2
T2 = Conv2D(filters=48, kernel_size=(1,1), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(T1)
# #T2 = ZeroPadding2D(padding=1)(T2)
T2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(T2)
if Dropout_active:
  T2 = Dropout(DO_val)(T2)
# 3
T3 = Conv2D(filters=1, kernel_size=(5,5), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(T2)
# #T3 = ZeroPadding2D(padding=1)(T3)
T3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(T3)

print(T3.shape)

# CNN_G

# 1
G1 = Conv2D(filters=96, kernel_size=(9,9), activation=activFunc,padding='same', kernel_initializer='glorot_normal', bias_initializer='zeros')(input_layer_guide)
#G1 = ZeroPadding2D(padding=1)(G1)
G1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(G1)

if Dropout_active:
  G1 = Dropout(DO_val)(G1)
# 2
G2 = Conv2D(filters=48, kernel_size=(1,1), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(G1)
#G2 = ZeroPadding2D(padding=1)(G2)
G2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(G2)
if Dropout_active:
  G2 = Dropout(DO_val)(G2)
# 3
G3 = Conv2D(filters=1, kernel_size=(5,5), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(G2)
# G3 = ZeroPadding2D(padding=1)(G3)
G3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(G3)


print(G3.shape)
concatenationLayer = concatenate([T3, G3], axis=-1)

# CNN_F

# 1
F1 = Conv2D(filters=64, kernel_size=(9,9), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(concatenationLayer)				   
F1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(F1)
# F1 = Dropout(DO_val)(F1)
# 2
F2 = Conv2D(filters=32, kernel_size=(1,1), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(F1)
F2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(F2)
# F2 = Dropout(DO_val)(F2)

# 3
F3 = Conv2D(filters=1, kernel_size=(5,5), activation=activFunc,padding='same',kernel_initializer='glorot_normal', bias_initializer='zeros')(F2)
F3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                       beta_initializer='zeros', gamma_initializer='ones', 
                       moving_mean_initializer='zeros', 
                       moving_variance_initializer='ones', 
                       beta_regularizer=None, gamma_regularizer=None, 
                       beta_constraint=None, gamma_constraint=None)(F3)

print(F3.shape)
# Final Merging
merging = concatenate([F3, input_layer_target], axis=-1)

## DROPOUT
if Dropout_active:
  merging = Dropout(DO_val)(merging)

# output
output_layer = Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid', 
                       kernel_regularizer = l2(0.0001),  use_bias=False, 
                       kernel_initializer='he_normal')(merging)

                                                         
model =  Model([input_layer_target, input_layer_guide],output_layer)
model.summary() # Print model


(?, 64, 64, 1)
(?, 64, 64, 3)


/usr/local/lib/python3.6/dist-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as ELU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


(?, 64, 64, 1)
(?, 64, 64, 1)
(?, 64, 64, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 64, 64, 96)   7872        input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 64, 64, 96)   23424       input_10[0][0]                   
________________________________________________________________

#Train\Load weights





In [19]:
model.compile(optimizer=Adam(), loss="mse" ,metrics=[ 'acc'])
weight_saver = ModelCheckpoint(path_save_weight_checkPoint, monitor='val_acc', 
                                              save_best_only=True, save_weights_only=True)


early_stopping = EarlyStopping(monitor='loss', mode="min", patience=5)
if not predict:
  H=model.fit([T_train,G_train],Y_train,validation_data=([T_test,G_test],Y_test),epochs=1, shuffle=True,callbacks = [weight_saver, early_stopping])
  model.save_weights(path_save_weight)
if predict:
  model.load_weights(path_load_weight)


Train on 50000 samples, validate on 5000 samples
Epoch 1/1
50000/50000 [==============================] - 198s 4ms/step - loss: 4.6331e-04 - acc: 1.3524e-04 - val_loss: 7.1421e-04 - val_acc: 1.2451e-04


#Create Image

In [16]:

#Read Guide image
prev_error=0
post_error=0
num_images=1
for ind in range(num_images):
 
  Guide=np.full((1,540, 960, 3),0).astype('float32')
  temp=np.load( GuidePath2Predict)['arr_0']
  Guide[0,:,:,:]=temp[ind,:,:,:]
#Read Target image

  target=np.full((1,540, 960,1),0).astype('float32')
  temp=np.load( TargetPath2Predict)['arr_0']
  target[0,:,:,:]=temp[ind,:,:,:]

 
  Disp=np.full((1,540, 960, 1),0).astype('float32')
  temp=np.load( GT2Predict)['arr_0']
  Disp[0,:,:,:]=temp[ind,:,:,:]
  
  Output=np.full((540, 960, 3),0).astype('float32')
  TargetPatch=np.full((1,64, 64, 1),0).astype('float32')
  GuidePatch=np.full((1,64, 64, 3),0).astype('float32')
  for i in range(960):
    for j in range(540):
  
      if j-31>0 and i-31>0 and j<target.shape[1]-31 and i<target.shape[2]-31 and (j%16)==0 and (i%16)==0:
        TargetPatch[0,:,:,0]=target[0,j-32:j+32, i-32:i+32,0]/np.max(target)

        GuidePatch[0,:,:,:]=Guide[0,j-32:j+32, i-32:i+32,0:3]/np.max(Guide)

        predictPatch=model.predict([TargetPatch,GuidePatch])

        Output[j-16:j+16, i-16:i+16,0]=predictPatch[0,16:48,16:48,0]
  
  

  A=Disp[0,:,:,0]
  B=1-A
  B =  np.clip(B, 0.0, 1.0)
  B *= 255
  GT16=B
  GT16=GT16[32:508,32:928]
  GT16=GT16-np.min(GT16)
  GT16=(GT16/np.max(GT16))*254 +1
  T=target[0,32:508,32:928,0]

  T=((T-np.min(T)))

  T=(T/np.max(T))*254 +1
  area=GT16.shape[0]*GT16.shape[1]
  AbsDis=np.abs(T-GT16)
  errDis=AbsDis
  Error_T=np.sum(errDis)/area
  prev_error=prev_error+Error_T/num_images
  print(ind)
  print('error before JF')
  print(np.sum(np.abs(T-GT16))/area)

  P=Output[32:508,32:928,0]
  P=((P-np.min(P)))
  P=(P/np.max(P))*254 +1
  AbsDis=np.abs(P-GT16)
  errDis=AbsDis
  Error_O=np.sum(errDis)/area
  post_error=post_error+Error_O/num_images
  print('error after JF')
  print(np.sum(np.abs(P-GT16))/area)

print('//////////////////////////')  
print(GT16.shape[1])
print(GT16.shape[0])
print('-----------error before JF')
print(prev_error)
print('-----------error after JF')
print(post_error)
print('//////////////////////////')

0
error before JF
6.879185267857143
error after JF
2.542625546312275
//////////////////////////
896
476
-----------error before JF
6.879185267857143
-----------error after JF
2.542625546312275
//////////////////////////
